In [26]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32).flatten()
X_test = np.asarray(X_test, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.int32).flatten()

num_training=49000
num_validation=1000
num_test=10000


mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

#print(X_train[0].shape)

mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
X_train = (X_train - mean_pixel) / std_pixel
X_val = (X_val - mean_pixel) / std_pixel
X_test = (X_test - mean_pixel) / std_pixel

In [27]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

imgbatch, labels = next(iter(train_ds))
(imgbatch[:].shape)

TensorShape([64, 32, 32, 3])

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation = None)
        self.relu1 = tf.keras.layers.ReLU()
        self.drop1 = tf.keras.layers.Dropout(rate = 0.5)
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation = None)
        self.relu2 = tf.keras.layers.ReLU()
        self.drop2 = tf.keras.layers.Dropout(rate = 0.5)
        self.conv3 = tf.keras.layers.Conv2D(128, 3, activation = None)
        self.relu3 = tf.keras.layers.ReLU()
        self.drop3 = tf.keras.layers.Dropout(rate = 0.5)

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(64, activation = 'relu')
        self.fc2 = tf.keras.layers.Dense(64, activation = 'relu')
        self.fc3 = tf.keras.layers.Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        #x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.conv2(x)
        #x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.drop3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)
        
model = MyModel()
predictions = model(imgbatch)

In [29]:
print(predictions[:5])

tf.Tensor(
[[ 0.01723444 -0.0636257  -0.00909126  0.0579405  -0.0164162   0.07487182
  -0.12164606  0.05273282 -0.00922599  0.1078472 ]
 [-0.06413941  0.03288643 -0.00937481 -0.02069273  0.04619693 -0.02512129
  -0.07923211  0.03037092 -0.00762598  0.19274282]
 [-0.06577777 -0.08515915 -0.01229723 -0.10129666  0.04464285 -0.01058963
  -0.03122757  0.07493231 -0.1626547   0.2544134 ]
 [-0.03193346  0.0039906   0.05535747 -0.02322285  0.02777594 -0.00977559
  -0.07404755  0.03001895 -0.00465857  0.15688552]
 [-0.05750206 -0.01129444  0.02265252 -0.01777416 -0.00962817 -0.04331404
  -0.02601763 -0.03231688 -0.07471838  0.11049508]], shape=(5, 10), dtype=float32)


In [30]:
tf.nn.softmax(predictions[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.10061045, 0.0927953 , 0.09799638, 0.10479041, 0.09728118,
        0.10657975, 0.0875645 , 0.10424611, 0.09798317, 0.11015285],
       [0.09264232, 0.10208154, 0.09785733, 0.09675603, 0.10344937,
        0.0963285 , 0.09125459, 0.10182507, 0.09802862, 0.11977659],
       [0.09393518, 0.09213212, 0.09909565, 0.09065727, 0.1049019 ,
        0.09926502, 0.09723739, 0.10812794, 0.08526194, 0.12938549],
       [0.09543243, 0.09892307, 0.10413722, 0.09626734, 0.1013042 ,
        0.09757062, 0.09149684, 0.10153168, 0.09807117, 0.11526542],
       [0.09562098, 0.10014307, 0.10360099, 0.09949628, 0.10031008,
        0.09698733, 0.09867945, 0.0980598 , 0.09398884, 0.11311324]],
      dtype=float32)>

In [31]:
print("Prcitions: {}".format(tf.argmax(predictions[:5], axis = 1)))
print("   Labels: {}".format(labels[:5]))

Prcitions: [9 9 9 9 9]
   Labels: [8 8 1 5 2]


In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

In [33]:
y_ = model(imgbatch, training = False)
l = loss_object(y_true = labels, y_pred = y_)
print("Loss Example: {}".format(l))

Loss Example: 2.303851842880249


In [34]:
print("Eager execution: {}".format(tf.executing_eagerly()))

Eager execution: True


In [35]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    #print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(), loss.numpy()))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [25]:
train_step(imgbatch, labels)
train_step(imgbatch, labels)
train_step(imgbatch, labels)
train_step(imgbatch, labels)
train_step(imgbatch, labels)
train_step(imgbatch, labels)

In [36]:
@tf.function
def val_step (images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    val_loss(v_loss)
    val_accuracy(labels, predictions)

In [ ]:
train_loss_results = []
train_accuracy_results = []
EPOCHS = 15

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for val_images, val_labels in val_ds:
        val_step(val_images, val_labels)
        
    
    template = 'Epoch {}, Train Loss: {}, Train Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          val_loss.result(),
                          val_accuracy.result()*100))
    train_loss_results.append(train_loss.result())
    train_accuracy_results.append(train_accuracy.result())

In [38]:
import matplotlib.pyplot as plt
fig, axes = plt.subplot(2, sharex=True, figsize=(12, 8))
fig.subtitle('Trainig Metrics')

axes[0].set_ylabel("Loss", fontsize = 12)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy" , fontsize = 12)
axes[1].set_xlabel("Epoch", fontsize = 12)
axes[1].plot(train_accuracy_results)

plt.show()

ValueError: Integer subplot specification must be a three-digit number, not 2

<Figure size 432x288 with 0 Axes>